In [3]:
!pip install sklearn-pandas

  Using cached https://files.pythonhosted.org/packages/c5/9e/42d7bcddb09a3ff52d0c60c810ba5d0fded28abbe320c85bbf7368192956/sklearn_pandas-1.6.0-py2.py3-none-any.whl


In [4]:
from datascience import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
plt.style.use('ggplot')

# import various tools from `sklearn`
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import sklearn.cluster as skcluster
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

from sklearn_pandas import DataFrameMapper
from collections import Counter

# Initial forays into the canonicalization problem

As with any problem where you start off having no idea what to do, exploratory data analysis is required.

In [12]:
dat = pd.read_json('../data/test_data.json')
metadata = pd.DataFrame(list(dat['metadata']))
metadata

,annotators,article_number,city,date_published,filename,periodical,periodical_code,state,version
0,[CRV],1368,Charleston,2011-11-23,"1368Charleston, SC-live5news-03.txt",live5news,03,SC,27
1,[NBA],7331,Seattle,2011-10-29,7331Seattle-King5_Seattle-01.txt,King5_Seattle,01,None,30
2,"[CRV, EDC]",1008,Bethlehem,2011-12-08,"1008Bethlehem, PA-theassociatedpressstatelocal...",theassociatedpressstatelocalwire,04,PA,26
3,"[CD, V]",1767,Cleveland,None,"1767Cleveland, TN-ClevelandBanner-01.txt",ClevelandBanner,01,TN,None
4,"[EDC, MB]",912,Baltimore,2011-12-05,912Baltimore-TheBaltimoreSun-01.txt,TheBaltimoreSun,01,None,26
5,[SLG],202,Albany,2011-12-23,"202Albany, NY-TimesUnion-04.txt",TimesUnion,04,NY,26
6,[CRV],2335,Denver,2011-12-12,2335Denver-theassociatedpressstatelocalwire-20...,theassociatedpressstatelocalwire,20,None,31
7,[CRV],3023,Huntington,2011-12-01,3023Huntington-theassociatedpressstatelocalwir...,theassociatedpressstatelocalwire,03,None,28
8,[SLG],4008,Lubbock,2011-11-15,4008Lubbock-KBCD-03.txt,KBCD,03,None,29
9,[CRV],3638,LosAngeles,2011-12-03,3638LosAngeles-LATimes-05.txt,LATimes,05,None,29


In [13]:
dat

,article_number,batch_name,highlight_tasks,id,metadata,sha256,text,uri
0,1368,article,"[{'id': 19, 'uuid': '6f192d04-fb08-456a-9a1b-e...",19,"{'city': 'Charleston', 'article_number': '1368...",a4a5ac25b071cf39c08e59130edadd9dd5d3e2f3327146...,Occupy demonstrators to be released on bond\nP...,urn:article:1368:7f77fa37-5e90-40f4-9773-6e6c2...
1,7331,article,"[{'id': 188, 'uuid': '7bcd739e-10b2-4c1c-b2fc-...",193,"{'city': 'Seattle', 'article_number': '7331', ...",39483cd1cae6a542d16fd3aca102933aa0d467d8f89a00...,by NATASHA RYAN / KING 5 News\nBio | Email \nP...,urn:article:7331:1bb03387-2ddd-4da6-a8a4-8630d...
2,1008,article,"[{'id': 2, 'uuid': '5c3baf23-8e4f-4b63-98d0-3d...",2,"{'city': 'Bethlehem', 'article_number': '1008'...",e1b90ef7acec907bdd37a0fe2131197b80f3e2cc6e1d7e...,3 of 4 DOCUMENTS\nThe Associated Press State &...,urn:article:1008:66b617ae-3214-4793-b0e4-bc66b...
3,1767,article,"[{'id': 28, 'uuid': 'ab33fe6b-bfa0-4708-87a0-9...",28,"{'city': 'Cleveland', 'article_number': '1767'...",c530764418bbc80089713af0120ef8d374415826f3b232...,6 protesters form Occupy Cleveland\nby DAVID D...,urn:article:1767:69fcd39c-ca01-4f66-939c-fadad...
4,912,article,"[{'id': 215, 'uuid': '4b4dd3ad-1e88-41d7-85e5-...",220,"{'city': 'Baltimore', 'article_number': '912',...",3744198c4950b0eb90bb4de5cf3120299ee0edc54b6af3...,"Woman stabbed at Occupy Baltimore encampment, ...",urn:article:912:646cf591-5716-43bb-bd21-86d961...
5,202,article,"[{'id': 34, 'uuid': 'e6649d19-8be3-4ca0-a7cd-9...",34,"{'city': 'Albany', 'article_number': '202', 'f...",c8b83b753ac022dae25614838cc55bd6feb4ca5a84ab19...,"Dismay, hard feelings after confrontation\nCit...",urn:article:202:ba3b47ff-99e4-4f0e-8ef8-0b05a9...
6,2335,article,"[{'id': 43, 'uuid': 'd1764b0a-7c2f-4567-84d5-7...",43,"{'city': 'Denver', 'article_number': '2335', '...",99186df03a5b919156b21cffdeec8af73bac2ee27205e7...,134 of 156 DOCUMENTS\nThe Associated Press Sta...,urn:article:2335:95127362-29ff-4e73-9329-42b67...
7,3023,article,"[{'id': 65, 'uuid': '1127c91e-3b3a-42c1-a6ba-5...",65,"{'city': 'Huntington', 'article_number': '3023...",553784be0adc747bf92bc969d3e589f59e169d82ae5688...,3 of 6 DOCUMENTS\nThe Associated Press State &...,urn:article:3023:5beeb6b3-6f37-4204-be9e-90a53...
8,4008,article,"[{'id': 97, 'uuid': '581589d3-13ee-410e-b92a-a...",97,"{'city': 'Lubbock', 'article_number': '4008', ...",a3f16dace56d26eca8af72dc25b0f732ed772a1d94b195...,KBCD: http://www.kcbd.com/\nMayor says Occupy ...,urn:article:4008:3d303fe3-eaff-46c7-89a8-d0e29...
9,3638,article,"[{'id': 84, 'uuid': '09dc4d57-e574-4062-bb06-7...",84,"{'city': 'LosAngeles', 'article_number': '3638...",ffa72b44d42511726447abe5cee341c1007cd280947b0d...,Occupy L.A. protesters return to downtown stre...,urn:article:3638:1c913dfa-9041-42a6-af1e-8f791...


In [6]:
city_and_date = pd.DataFrame(list(zip(metadata['city'], metadata['date_published'])))
text_and_id = pd.DataFrame(list(zip(dat['id'], dat['text'])))
concat_tbl = pd.concat([city_and_date, text_and_id], axis = 1) 
concat_tbl.columns = ['city', 'date', 'id', 'text']

In [7]:
reading_set = Table().from_df(concat_tbl)
reading_set.to_csv("reading_set.csv")

In [8]:
charleston = reading_set.where("city", "Charleston")

In [9]:
event_ids = make_array(1, 2, 3, 4, 5)
charleston = charleston.with_column('event_ids', event_ids)

In [10]:
reading_set.group("city").sort("count").show()

city,count
Allentown City,1
Anchorage,1
Bangor,1
Baton Rouge,1
Bethlehem,1
Bloomington,1
Charlotte,1
Chicago,1
Cleveland,1
Detroit,1


In [11]:
reading_set

city,date,id,text
Charleston,2011-11-23,19,Occupy demonstrators to be released on bond Posted: Nov ...
Seattle,2011-10-29,193,by NATASHA RYAN / KING 5 News Bio | Email Posted on Oct ...
Bethlehem,2011-12-08,2,3 of 4 DOCUMENTS The Associated Press State & Local Wire ...
Cleveland,None,28,"6 protesters form Occupy Cleveland by DAVID DAVIS, Manag ..."
Baltimore,2011-12-05,220,"Woman stabbed at Occupy Baltimore encampment, police say ..."
Albany,2011-12-23,34,"Dismay, hard feelings after confrontation City leaders d ..."
Denver,2011-12-12,43,134 of 156 DOCUMENTS The Associated Press State & Local ...
Huntington,2011-12-01,65,3 of 6 DOCUMENTS The Associated Press State & Local Wire ...
Lubbock,2011-11-15,97,KBCD: http://www.kcbd.com/ Mayor says Occupy Lubbock pro ...
LosAngeles,2011-12-03,84,Occupy L.A. protesters return to downtown streets Decemb ...
